# PART 1: IDENTIFICATION (Python)
FIR & ARX with Least Squares + MDL (mirrors the MATLAB-style workflow).


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sysid.hankel import hankel_fir, hankel_arx
from sysid.ls import ls_fit
from sysid.metrics import mse_loss, mdl_score, nrmse
from sysid.utils import autocorr, train_valid_split
%matplotlib inline


In [ ]:
df = pd.read_csv('../data/sample_timeseries.csv')
u, y = df['u'].values, df['y'].values
plt.figure(figsize=(10,3)); plt.plot(u[:200]); plt.title('Input u (first 200)'); plt.grid(True)
plt.figure(figsize=(10,3)); plt.plot(y[:200]); plt.title('Output y (first 200)'); plt.grid(True)


In [ ]:
ac = autocorr(y, max_lag=30)
plt.figure(); plt.stem(range(len(ac)), ac, use_line_collection=True)
plt.title('AutoCorrelation of y'); plt.xlabel('Lag'); plt.ylabel('rho'); plt.grid(True)


In [ ]:
u_tr, y_tr, u_va, y_va = train_valid_split(u, y, 0.8)
len(y_tr), len(y_va)


In [ ]:
max_order = 20
loss_fir, mdl_fir = [], []
for n in range(1, max_order+1):
    Phi = hankel_fir(u_tr, n)
    target = y_tr[n:]
    theta, yhat = ls_fit(Phi, target)
    J = mse_loss(target, yhat)
    loss_fir.append(J)
    mdl_fir.append(mdl_score(J, len(y_tr)-n, n))
best_fir = int(np.argmin(mdl_fir)+1)
best_fir


In [ ]:
Phi_fir = hankel_fir(u_tr, best_fir)
theta_fir, yhat_tr_fir = ls_fit(Phi_fir, y_tr[best_fir:])
Phi_fir_v = hankel_fir(u_va, best_fir)
yhat_va_fir = Phi_fir_v @ theta_fir
nrmse_fir = nrmse(y_va[best_fir:], yhat_va_fir)
nrmse_fir


In [ ]:
loss_arx, mdl_arx = [], []
for n in range(1, max_order+1):
    Phi = hankel_arx(u_tr, y_tr, n)
    target = y_tr[n:]
    theta, yhat = ls_fit(Phi, target)
    J = mse_loss(target, yhat)
    loss_arx.append(J)
    mdl_arx.append(mdl_score(J, len(y_tr)-n, 2*n))
best_arx = int(np.argmin(mdl_arx)+1)
best_arx


In [ ]:
# Mark minimum MDL points for FIR & ARX (if arrays exist)
import numpy as np
min_idx_fir = int(np.argmin(mdl_fir)+1); min_val_fir = float(np.min(mdl_fir))
plt.figure(); plt.plot(range(1, max_order+1), mdl_fir, '-o');
plt.scatter([min_idx_fir],[min_val_fir]); plt.text(min_idx_fir, min_val_fir, f" min MDL={min_idx_fir}")
plt.title('Order Estimation by MDL (FIR)'); plt.xlabel('order n'); plt.ylabel('MDL(n)'); plt.grid(True)

min_idx_arx = int(np.argmin(mdl_arx)+1); min_val_arx = float(np.min(mdl_arx))
plt.figure(); plt.plot(range(1, max_order+1), mdl_arx, '-o');
plt.scatter([min_idx_arx],[min_val_arx]); plt.text(min_idx_arx, min_val_arx, f" min MDL={min_idx_arx}")
plt.title('Order Estimation by MDL (ARX)'); plt.xlabel('order n'); plt.ylabel('MDL(n)'); plt.grid(True)


In [ ]:
Phi_arx = hankel_arx(u_tr, y_tr, best_arx)
theta_arx, yhat_tr_arx = ls_fit(Phi_arx, y_tr[best_arx:])
Phi_arx_v = hankel_arx(u_va, y_va, best_arx)
yhat_va_arx = Phi_arx_v @ theta_arx
nrmse_arx = nrmse(y_va[best_arx:], yhat_va_arx)
nrmse_arx


In [ ]:
k = range(len(y_va))
plt.figure(figsize=(10,4))
plt.plot(y_va, label='y_valid')
plt.plot(list(k)[best_fir:], yhat_va_fir, label=f'FIR(n={best_fir})')
plt.plot(list(k)[best_arx:], yhat_va_arx, label=f'ARX(n={best_arx})')
plt.legend(); plt.grid(True); plt.title('Validation Fit')
print('NRMSE - FIR:', nrmse_fir)
print('NRMSE - ARX:', nrmse_arx)


## Whiteness & Cross-correlation tests (ARX residuals)

In [ ]:
eps = y_va[best_arx:] - yhat_va_arx
from sysid.utils import autocorr as _ac, crosscorr
ac_eps = _ac(eps, max_lag=30)
plt.figure(); plt.stem(range(len(ac_eps)), ac_eps, use_line_collection=True)
plt.title('AUTOCORRELATION OF ERRORS'); plt.xlabel('Lag'); plt.ylabel('rho'); plt.ylim(-1,1); plt.grid(True)


In [ ]:
lags, cc = crosscorr(eps, u_va[best_arx:], max_lag=30)
plt.figure(); plt.stem(lags, cc, use_line_collection=True)
plt.title('Cross-correlation: residuals vs input (validation)'); plt.xlabel('lag'); plt.ylabel('R_eu(lag)'); plt.ylim(-0.5,0.5); plt.grid(True)


In [ ]:
k = range(len(y_va))
plt.figure();
plt.plot(list(k)[best_arx:], y_va[best_arx:], label='Y validation')
plt.plot(list(k)[best_arx:], yhat_va_arx, label='Y predicted')
plt.legend(); plt.grid(True); plt.title('Validation vs Predicted (ARX)')


# PART 2: CLASSIFICATION — Logistic Regression with Newton's Method

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from classify.newton_logreg import fit_ovr_newton, predict_proba_ovr
dfc = pd.read_csv('../data/sample_classification.csv')
X = dfc[['x1','x2']].values; y = dfc['y'].values.astype(int)
m = len(y); ntr = int(0.8*m)
Xtr, ytr = X[:ntr], y[:ntr]
Xte, yte = X[ntr:], y[ntr:]
plt.figure(); plt.scatter(Xtr[:,0], Xtr[:,1], c=ytr, marker='o'); plt.title('Original representation of data (train)'); plt.grid(True)


## Train with Newton's Method

In [ ]:
Phi_tr = np.hstack([np.ones((Xtr.shape[0],1)), Xtr])
theta, histories, classes = fit_ovr_newton(Phi_tr, ytr, max_iter=40, lr=1.0)
plt.figure(); plt.plot(range(1, len(histories[-1])+1), histories[-1], '-o');
plt.title('Loss function J(theta) for one-vs-rest (one class)'); plt.xlabel('iteration'); plt.ylabel('loss'); plt.grid(True)


## Validation & Analysis

In [ ]:
Phi_te = np.hstack([np.ones((Xte.shape[0],1)), Xte])
P_tr = predict_proba_ovr(Phi_tr, theta)
P_te = predict_proba_ovr(Phi_te, theta)
yhat_tr = P_tr.argmax(axis=1)
yhat_te = P_te.argmax(axis=1)
acc_tr = (yhat_tr==ytr).mean(); acc_te=(yhat_te==yte).mean()
print('Training acc %:', round(acc_tr*100,2), ' Test acc %:', round(acc_te*100,2))
plt.figure(); plt.scatter(Xte[:,0], Xte[:,1], c=yhat_te, marker='o'); plt.title("Predicted classes (test)"); plt.grid(True)


In [ ]:
mis = yhat_te != yte
plt.figure();
plt.scatter(Xte[~mis,0], Xte[~mis,1], marker='o');
plt.scatter(Xte[mis,0], Xte[mis,1], marker='x');
plt.title("'o' correct, 'x' misclassified (test)"); plt.grid(True)
